# Final Model TESTİNG

In [1]:
# === PATHS (3rd stage best) ===
from pathlib import Path
from ultralytics import YOLO
import torch

BEST_PT = Path(r"C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\third_stage\runs\best_finetune_img896_ep30.pt")
DATA_YAML = Path(r"C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\scut_head_yolo_rebalanced\dataset.yaml")

assert BEST_PT.exists(), f"best.pt bulunamadı: {BEST_PT}"
assert DATA_YAML.exists(), f"dataset.yaml bulunamadı: {DATA_YAML}"

print("BEST_PT:", BEST_PT)
print("DATA_YAML:", DATA_YAML)
print("CUDA:", torch.cuda.is_available(), "| GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else None)

BEST_PT: C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\third_stage\runs\best_finetune_img896_ep30.pt
DATA_YAML: C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\scut_head_yolo_rebalanced\dataset.yaml
CUDA: True | GPU: NVIDIA GeForce RTX 3060 Laptop GPU


In [2]:
model = YOLO(str(BEST_PT))

test_det = model.val(
    data=str(DATA_YAML),
    split="test",     # kritik
    imgsz=896,
    conf=0.34,
    iou=0.45,
    device=0,
    verbose=True
)

test_det

Ultralytics 8.3.203  Python-3.10.18 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 27.516.8 MB/s, size: 66.8 KB)
val: Scanning C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\scut_head_yolo_rebalanced\labels\test... 700 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 700/700 365.2it/s 1.9s0.1s
val: New cache created: C:\Users\hdgn5\OneDrive\Masast\Head Detection\scut_head_yolo_rebalanced\labels\test.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 44/44 3.1it/s 14.0s0.3s
                   all        700      18605      0.935      0.941      0.957       0.55
Speed: 0.8ms preprocess, 10.5ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to C:\Users\hdgn5\OneDrive\Masast\Head Detection\third_stage\runs\detect\val2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001F28E092C20>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [3]:
import glob
import numpy as np
import pandas as pd
import yaml

def count_gt_from_label(label_path: Path) -> int:
    if not label_path.exists():
        return 0
    with open(label_path, "r", encoding="utf-8") as f:
        lines = [ln.strip() for ln in f.readlines() if ln.strip()]
    return len(lines)

def safe_relpath(p: Path, root: Path) -> str:
    try:
        return str(p.relative_to(root))
    except Exception:
        return str(p)

with open(DATA_YAML, "r", encoding="utf-8") as f:
    dcfg = yaml.safe_load(f)

test_images_root = Path(DATASET_DIR := Path(DATA_YAML).parent) / dcfg["test"] 
if Path(dcfg["test"]).is_absolute():
    test_images_root = Path(dcfg["test"])

img_exts = ("*.jpg","*.jpeg","*.png","*.bmp","*.webp")
image_paths = []
for ext in img_exts:
    image_paths += list(test_images_root.rglob(ext))
image_paths = sorted(image_paths)

print("Test images root:", test_images_root)
print("N_test_images:", len(image_paths))

labels_root_guess = Path(str(test_images_root).replace("\\images\\", "\\labels\\").replace("/images/", "/labels/"))
print("Labels root guess:", labels_root_guess)


Test images root: C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\scut_head_yolo_rebalanced\images\test
N_test_images: 700
Labels root guess: C:\Users\hdgn5\OneDrive\Masaüstü\Head Detection\scut_head_yolo_rebalanced\labels\test


In [4]:
from math import sqrt

CONF = 0.34
IOU  = 0.45
IMGSZ = 896

rows = []

for img_path in image_paths:

    rel = img_path.relative_to(test_images_root)
    label_path = labels_root_guess / rel.with_suffix(".txt")

    gt = count_gt_from_label(label_path)

    pred = model.predict(
        source=str(img_path),
        imgsz=IMGSZ,
        conf=CONF,
        iou=IOU,
        device=0,
        verbose=False
    )[0]

    pred_count = 0 if pred.boxes is None else int(len(pred.boxes))

    rows.append({
        "image": str(img_path),
        "gt": gt,
        "pred": pred_count,
        "err": pred_count - gt,
        "abs_err": abs(pred_count - gt)
    })

df = pd.DataFrame(rows)

N_images = len(df)
GT_total = int(df["gt"].sum())
Pred_total = int(df["pred"].sum())
Abs_total_diff = int(abs(Pred_total - GT_total))
MAE = float(df["abs_err"].mean())
RMSE = float(sqrt((df["err"]**2).mean()))

mask = df["gt"] > 0
MAPE = float((df.loc[mask, "abs_err"] / df.loc[mask, "gt"]).mean()) if mask.any() else float("nan")
Bias = int(Pred_total - GT_total)

summary = {
    "conf": CONF,
    "iou": IOU,
    "imgsz": IMGSZ,
    "N_images": N_images,
    "GT_total": GT_total,
    "Pred_total": Pred_total,
    "Abs_total_diff": Abs_total_diff,
    "MAE": MAE,
    "RMSE": RMSE,
    "MAPE(ignore gt=0)": MAPE,
    "bias(Pred-GT)": Bias
}
summary

{'conf': 0.34,
 'iou': 0.45,
 'imgsz': 896,
 'N_images': 700,
 'GT_total': 18605,
 'Pred_total': 18852,
 'Abs_total_diff': 247,
 'MAE': 1.6328571428571428,
 'RMSE': 4.763402145525822,
 'MAPE(ignore gt=0)': 0.07192163901806672,
 'bias(Pred-GT)': 247}

In [5]:
df.sort_values("abs_err", ascending=False).head(25)

,image,gt,pred,err,abs_err
668,C:\Users\hdgn5\OneDrive\Masaüstü\Head Detectio...,93,172,79,79
368,C:\Users\hdgn5\OneDrive\Masaüstü\Head Detectio...,54,115,61,61
660,C:\Users\hdgn5\OneDrive\Masaüstü\Head Detectio...,62,89,27,27
627,C:\Users\hdgn5\OneDrive\Masaüstü\Head Detectio...,58,37,-21,21
480,C:\Users\hdgn5\OneDrive\Masaüstü\Head Detectio...,46,66,20,20
495,C:\Users\hdgn5\OneDrive\Masaüstü\Head Detectio...,76,57,-19,19
198,C:\Users\hdgn5\OneDrive\Masaüstü\Head Detectio...,60,75,15,15
174,C:\Users\hdgn5\OneDrive\Masaüstü\Head Detectio...,62,77,15,15
413,C:\Users\hdgn5\OneDrive\Masaüstü\Head Detectio...,11,23,12,12
199,C:\Users\hdgn5\OneDrive\Masaüstü\Head Detectio...,66,78,12,12
